In [ ]:
require 'csv'
require 'redis'
require 'json'
require 'date'

#1 year version (2017)
INPUTDATA = 'input_data_1y.txt'


#10 years version (2008-2017)
#INPUTDATA = 'input_data_10y.txt'

redis = Redis.new(host: "redis", port: 6379, db: 0)
f = File.open(INPUTDATA, 'r')
f.readline
redis.flushdb
key = 0
begin
  loop do
    
    if key % 10000 == 0 then puts "#{Time.now} insert-data: #{key} processed" end

    redis.set(key,f.readline.strip.split("\t").map{|i| i.strip}.to_s)
    key = key + 1
  end  
rescue StandardError => e
  puts e
end  
size = key


(0..size - 1).each do |key|
  
  if key % 10000 == 0 then puts "#{Time.now} prr-params: #{key} processed" end
  
  t = redis.get(key).nil? ? nil : eval(redis.get(key))
  if t
    year, month = Date.parse(t[0]).year, Date.parse(t[0]).month
    drug, soc = t[1], t[2]
    redis.incr({incremental: false, year: year, month: month, drug: drug, soc: soc})
    redis.incr({incremental: false, year: year, month: month, drug: drug})
    redis.incr({incremental: false, year: year, month: month, soc: soc})
    redis.incr({incremental: false, year: year, month: month})
    (year..2017).each do |y|
      start_month = year == y ? month : 1
      (start_month..12).each do |m|
        redis.incr({incremental: true, year: y, month: m, drug: drug, soc: soc})
        redis.incr({incremental: true, year: y, month: m, drug: drug})
        redis.incr({incremental: true, year: y, month: m, soc: soc})
        redis.incr({incremental: true, year: y, month: m})
      end  
    end  
  end
end    